In [ ]:
import cv2
import numpy as np
from numpy import log10, array, sqrt
from IPython.display import clear_output
from time import sleep
from google.colab.patches import cv2_imshow

In [ ]:
import glob

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation

In [ ]:
from astropy.io import fits

In [ ]:
from astropy.utils.data import get_pkg_data_filename

In [ ]:
from astropy.visualization import ImageNormalize, SqrtStretch

In [ ]:
from sunpy.io import read_file

In [ ]:
import sunpy.map
from sunpy.map import Map

In [ ]:
import skimage.transform

In [ ]:
# load the image file
image_file1 = get_pkg_data_filename('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Fido_Sunpy/LASCO/22400113.fts')

In [ ]:
image_file = get_pkg_data_filename('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Fido_Sunpy/LASCO/22400118.fts')

In [ ]:
fits.info(image_file)

In [ ]:
data, header = read_file(image_file)[0]
angle = header['CROTA2']
print(angle)  # -> -173.555

In [ ]:
image_data = fits.getdata(image_file, ext=0)

In [ ]:
image_data = (image_data)

In [ ]:
print(image_data.shape)

In [ ]:
image_data1 = fits.getdata(image_file1, ext=0)

In [ ]:
plt.imshow(image_data, origin='lower')

In [ ]:
diff_image=image_data1-image_data

In [ ]:
plt.imshow(diff_image, vmin=-50, vmax=50,  origin='lower')
plt.axis('off')

In [ ]:
height, width = image_data.shape

center = (width / 2, height/ 2)

# Perform the rotation
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(diff_image, M, (width, height))

plt.imshow(rotated, vmin=-50, vmax=50,  origin='lower')

In [ ]:
np.amax(diff_image)

**Function to Rotate Image**

In [ ]:
def rotate(image, angle, center = None, scale = 1.0):
    height, width = image.shape

    if center is None:
        center = (width / 2, height/ 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (width, height))

    return rotated

## Intensity Transformation Operations on Images

**Piecewise-Linear Transformation Functions –**

In [ ]:
# Function to map each intensity level to output intensity level.
def pixelVal(pix, r1, s1, r2, s2):
    if (0 <= pix and pix <= r1):
        return (s1 / r1)*pix
    elif (r1 < pix and pix <= r2):
        return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1
    else:
        return ((255 - s2)/(255 - r2)) * (pix - r2) + s2

**Power-Law (Gamma) Transformation –**

In [ ]:
def gammaVal(img):
  # Trying 4 gamma values.
  for gamma in [6]:
    
    # Apply gamma correction.
    gamma_corrected = np.array(255*(img / 255) ** gamma, dtype = 'uint8')

  return (gamma_corrected)

In [ ]:
np.amin(image_data)

In [ ]:
np.amax(image_data)

## Reading the .fts files from the folder



In [ ]:
image_filename = glob.glob('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Fido_Sunpy/LASCO_old/*.fts')

In [ ]:
# setting the background as None in the beginning
image_file1 = get_pkg_data_filename(image_filename[0])
image_data1 = fits.getdata(image_file1, ext=0)
height, width = image_data1.shape
if height ==512:
  I2 = skimage.transform.resize(image_data1,np.asarray(image_data1.shape)/1,\
                              order=3,preserve_range=True,anti_aliasing=True)
else:
  I2 = skimage.transform.resize(image_data1,np.asarray(image_data1.shape)/2,\
                              order=3,preserve_range=True,anti_aliasing=True)
bgframe = I2

In [ ]:
ii=1

plt.figure(1)
plt.clf()

# On-screen, things will be displayed at 80dpi regardless of what we set here
# This is effectively the dpi for the saved figure. We need to specify it,
# otherwise `savefig` will pick a default dpi based on your local configuration
dpi = 300

nf = len(image_filename)

for index in range(1, nf):
  image_file = get_pkg_data_filename(image_filename[index])
  image_data = fits.getdata(image_file, ext=0)

  # in the very first iteration of while loop our background (bgframe) will be None so need to provide the
  # frame as background
  if bgframe is None:
    bgframe = image_filename[0]

  height, width = image_data.shape
  if height ==512:
    I1 = skimage.transform.resize(image_data,np.asarray(image_data.shape)/1,\
                                order=3,preserve_range=True,anti_aliasing=True)
  else:
    I1 = skimage.transform.resize(image_data,np.asarray(image_data.shape)/2,\
                                order=3,preserve_range=True,anti_aliasing=True)

  data, header = read_file(image_file)[0]
  angle = header['CROTA2']


  height, width = I1.shape

  # What size does the figure need to be in inches to fit the image?
  figsize = width / float(dpi), height / float(dpi)


  # Create a figure of the right size with one axes that takes up the full figure
  fig = plt.figure(figsize=figsize)
  ax = fig.add_axes([0, 0, 1, 1])

  # Hide spines, ticks, etc.
  ax.axis('off')

##            # Display the image.
##            ax.imshow(I, interpolation='nearest')


  # Ensure we're displaying with square pixels and the right extent.
  # This is optional if you haven't called `plot` or anything else that might
  # change the limits/aspect.  We don't need this step in this case.
  ax.set(xlim=[0, width], ylim=[height, 0], aspect=1)

  dif_image=bgframe-I1
  
  rotated_dif_image = rotate(dif_image, angle, center = None, scale = 1.0)

  plt.imshow(rotated_dif_image, vmin=-400, vmax=100,  origin='lower')
  plt.axis('off')
  fig.savefig('/home/dev/Sun/Aditya_L1/day2'+str(ii) +'.png',dpi=dpi, transparent=True)
  ii=ii+1

In [ ]:
np.amin(dif_image)

## Function to create video

In [ ]:
def video_creation(input_file,output_file):
  img_array = []
  for filename in glob.glob(input_file +'/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

  out = cv2.VideoWriter(output_file +'/project2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)

  for i in range(len(img_array)):
    out.write(img_array[i])

  out.release()

Video creation from output enhanced images

In [ ]:
input_file = '/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Enhanced_Image/'
output_file = '/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_movie/'
video_creation(input_file,output_file)

## Function to display video

In [ ]:
%pip install imageio-ffmpeg

In [ ]:
import imageio
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(10,10))  #Display size specification

    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime


Display the created video

In [ ]:
video = imageio.mimread(output_file + '/project2.avi')  #Loading video
#video = [resize(frame, (256, 256))[..., :3] for frame in video]    #Size adjustment (if necessary)
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

In [ ]:
# load the video file
cap = cv2.VideoCapture('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_movie/project2.avi')

In [ ]:
# setting the background as None in the beginning
bgframe = None

In [ ]:
i=1
fig = plt.figure(figsize=(8,8))  #Display size specification

while True:
    # access the frame from the video
    success,img = cap.read()

    # check if we get the frame
    if success == True:
        # resizing the frame because there is no need to process a large size frame
        ##img = cv2.resize(img,(600,500))
        # change the bgr frame into grayscale  format
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Applying Gaussian Blur on the Frame heavily to avoid any change in pixels intensity that
        # may occur due to camera sensor as no camera can capture a 100% accurate image.
        gray = cv2.GaussianBlur(gray,(25,25),0)

        # in the very first iteration of while loop our background (bgframe) will be None so need to provide the
        # frame as background
        if bgframe is None:
            bgframe = gray

        # Compute the difference between the first  frame (background) and current frame
        # using simple subtraction
        # delta = |background- current frame(|
        frameDelta = cv2.absdiff(bgframe, gray)

        # if any pixel value in 'frameDelta' is less than 25, we discard the pixel and set it to black (i.e. background).
        # If it is greater than 25, we’ll set it to white (i.e. foreground)
        _,thresh = cv2.threshold(frameDelta,35,255,cv2.THRESH_BINARY)

        # creating a kernel of 7x7
        kernel = np.ones((7, 7), np.uint8)
        
        # dilating the frame
        thresh = cv2.dilate(thresh, None, iterations=6)
        # finding the contours
        contours, heirarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # finding area of contour
            area = cv2.contourArea(contour)
            print(area)
            # if area greater than the specified value the only then we will consider it
            if area > 1500:
                # find the rectangle co-ordinates
                x,y,w,h = cv2.boundingRect(contour)

                # and then draw it to indicate the moving object
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),3)
                cv2.putText(img,'MOTION DETECTED',(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)

        # displaying the frame
        ##cv2_imshow(img)
        #cv2.imwrite('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_frame/image'+str(i)+'.png', img)
        ##cv2_imshow(thresh)
        cv2.imwrite('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Threshold_frame/image'+str(i)+'.png', thresh)

        # Make the grey scale image have three channels
        grey_3_channel = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
        ##grey_3_channel = thresh
        numpy_horizontal = np.hstack((img, grey_3_channel))
        ##cv2_imshow(numpy_horizontal)

        clear_output()
        cv2_imshow(numpy_horizontal)
        #plt.show()
        sleep(0.5) #demo: it show the img even when halted

        ##plt.close()
        
        cv2.imwrite('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Horizontal/image'+str(i)+'.png', numpy_horizontal)
        cv2.waitKey(0)
        i=i+1
    else:
        break

Video creation from output horizontal images

In [ ]:
input_file = '/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Horizontal/'
output_file = '/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_movie/H/'
video_creation(input_file,output_file)

In [ ]:
cap=cv2.VideoCapture('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_movie/H/project2.avi')

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")
  
# Read until video is completed
while(cap.isOpened()):
      
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
    # Display the resulting frame
        ##cv2_imshow(frame)
          
    # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
  
# Break the loop
    else:
        break
  
# When everything done, release
# the video capture object
cap.release()

In [ ]:
video = imageio.mimread('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Resultant_movie/H/project2.avi')  #Loading video
#video = [resize(frame, (256, 256))[..., :3] for frame in video]    #Size adjustment (if necessary)
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

In [ ]:
cap.release()
# Closes all the frames
cv2.destroyAllWindows()

## Find the Center of a Blob (Centroid)

In [ ]:
# read image through command line
img = cv2.imread('/home/dev/Sun/Aditya_L1/day2/VSO_Data/Solar_Movie/Results/Threshold_frame/image5.png')

# convert image to grayscale image
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
# convert the grayscale image to binary image
ret,thresh = cv2.threshold(gray_image,127,255,0)
 
# calculate moments of binary image
M = cv2.moments(thresh)
 
# calculate x,y coordinate of center
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])

# put text and highlight the center
cv2.circle(img, (cX, cY), 5, (0,255,255), -1)
cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255, 255), 2)

width, height = img.shape[1], img.shape[0]
mid_x, mid_y = int(width/2), int(height/2)
print(mid_x)
print(mid_y)

#Draw center line
cv2.line(img, (0, mid_y), (width, mid_y), color=(255, 0, 255), thickness=2)
cv2.line(img, (mid_x, 0), (mid_x, height), color=(255, 0, 255), thickness=2)
 
# display the image
cv2_imshow(img)
cv2.waitKey(0)

In [ ]:
print(cX)
print(cY)


In [ ]:
# Import math Library
import math

In [ ]:
center = [mid_x, mid_y]
vector_1= [center[0], center[1]]
vector_2 = [cX, cY]
unit_vector_2 = vector_1 / np.linalg.norm(vector_1)
unit_vector_1 = vector_2 / np.linalg.norm(vector_2)
dot_product = np.dot(unit_vector_1, unit_vector_2)
angle = np.arccos(dot_product)
degrees = math.degrees(angle)
print(degrees)